# 🚀 LUMPS - Worker Distribuido para Google Colab

Este notebook permite ejecutar un worker distribuido para generar el dataset de la Fase 0 de LUMPS.

## 📋 Instrucciones

1. **"Runtime" → "Run all"** - ¡El notebook es completamente automático!
2. **El worker detectará automáticamente** si existe carpeta compartida
3. **El worker trabajará automáticamente** hasta que se desconecte Colab
4. **Puedes pausar/reanudar** usando los botones de control

## 🎯 Comportamiento Automático

- **Si existe carpeta compartida**: Se une al trabajo coordinado con otros workers
- **Si no existe carpeta compartida**: Crea carpeta local y trabaja independientemente
- **Detección inteligente**: No necesitas configurar nada manualmente

## ⚠️ Importante
- Cada usuario debe hacer **"Archivo > Guardar una copia en Drive"** antes de ejecutar
- El worker trabajará en **loop continuo** hasta desconexión
- Los resultados se guardan automáticamente en Google Drive


## 🔧 Setup Inicial


In [ ]:
# Montar Google Drive
from google.colab import drive
import os

print("🔗 Montando Google Drive...")
drive.mount('/content/drive')
print("✅ Google Drive montado exitosamente")


In [ ]:
# Clonar repositorio desde GitHub automáticamente
import subprocess
import sys
from pathlib import Path

print("📥 Clonando repositorio LUMPS...")

# Cambiar al directorio de trabajo
%cd /content

# URL del repositorio (actualizada automáticamente)
repo_url = "https://github.com/juanignaciorey/LUMPS.git"

# Verificar si ya existe el directorio
if Path("LUMPS").exists():
    print("📁 Repositorio ya existe, actualizando...")
    %cd LUMPS
    !git pull origin master
else:
    print("📥 Clonando repositorio...")
    !git clone $repo_url
    %cd LUMPS

print("✅ Repositorio listo")
print(f"📁 Directorio actual: {os.getcwd()}")


In [ ]:
# Instalar dependencias automáticamente
print("📦 Instalando dependencias...")

try:
    # Instalar requirements
    !pip install -r requirements.txt

    # Instalar dependencias adicionales para Colab
    !pip install tqdm

    print("✅ Dependencias instaladas exitosamente")
except Exception as e:
    print(f"⚠️ Error instalando dependencias: {e}")
    print("🔄 Intentando instalar dependencias básicas...")
    !pip install torch numpy tqdm
    print("✅ Dependencias básicas instaladas")


## ⚙️ Configuración


In [ ]:
# Configurar paths y variables automáticamente
import os
import socket
import time
from pathlib import Path

# En Google Colab, las carpetas compartidas aparecen en MyDrive cuando se crea acceso directo
MY_DRIVE_PATH = "/content/drive/MyDrive/LUMPS_Distributed"

print("🔍 Detectando carpeta compartida o acceso directo...")

# Verificar si existe la carpeta (puede ser local o acceso directo a compartida)
if Path(MY_DRIVE_PATH).exists():
    try:
        # Listar contenido para verificar si es un acceso directo funcional
        contents = os.listdir(MY_DRIVE_PATH)
        print(f"✅ Carpeta encontrada: {MY_DRIVE_PATH}")
        print(f"📁 Contenido: {contents[:5]}{'...' if len(contents) > 5 else ''}")

        # Verificar si tiene estructura de proyecto distribuido (indica carpeta compartida)
        has_manifests = any('manifest' in item.lower() for item in contents)
        has_results = any('result' in item.lower() for item in contents)
        has_logs = any('log' in item.lower() for item in contents)

        # Si tiene estructura de proyecto distribuido, es carpeta compartida
        if has_manifests or has_results or has_logs:
            print("🎯 Detectado acceso directo a carpeta compartida - modo coordinado")
            print("🤝 Múltiples workers coordinados trabajando juntos")
        else:
            print("🎯 Carpeta local detectada - worker independiente")
            print("💡 Para coordinación: crea acceso directo a carpeta compartida")

        DRIVE_BASE_PATH = MY_DRIVE_PATH

    except Exception as e:
        print(f"⚠️ Error accediendo a carpeta: {e}")
        DRIVE_BASE_PATH = MY_DRIVE_PATH
        print("🎯 Usando carpeta local - worker independiente")
else:
    # Crear carpeta local por defecto
    DRIVE_BASE_PATH = MY_DRIVE_PATH
    print("📁 No se encontró carpeta existente, creando carpeta local...")
    try:
        Path(DRIVE_BASE_PATH).mkdir(parents=True, exist_ok=True)
        print(f"✅ Carpeta local creada: {DRIVE_BASE_PATH}")
        print("💡 Para coordinación entre workers:")
        print("   1. Comparte la carpeta desde el master")
        print("   2. Crea acceso directo en tu Drive")
        print("   3. Re-ejecuta este notebook")
    except Exception as e:
        print(f"❌ Error creando carpeta: {e}")
        print("💡 Verifica que tienes permisos de escritura en Drive")

# Generar ID único para este worker
WORKER_ID = f"colab_{socket.gethostname()}_{os.getpid()}_{int(time.time())}"

# Configurar variables de entorno
os.environ['LUMPS_DRIVE_PATH'] = DRIVE_BASE_PATH
os.environ['WORKER_ID'] = WORKER_ID

print(f"\n🔧 Configuración automática:")
print(f"   Worker ID: {WORKER_ID}")
print(f"   Drive Path: {DRIVE_BASE_PATH}")
print(f"   Hostname: {socket.gethostname()}")
print(f"   PID: {os.getpid()}")

print("🎯 Configuración completada")


## 🔧 Setup Automático

El notebook configurará automáticamente todo lo necesario:
- Creará la carpeta compartida si no existe
- Generará el manifiesto de batches automáticamente
- Configurará el worker para ejecutar


### 🛡️ Mejoras de Resiliencia

El worker ahora incluye mejoras para manejar errores comunes:
- **Recuperación automática** cuando se pierde el manifiesto
- **Reintentos inteligentes** en caso de errores temporales
- **Logging mejorado** para diagnóstico de problemas
- **Manejo robusto** de problemas de sincronización con Google Drive

**Si ves errores de "Manifiesto no encontrado":**
- El worker intentará recuperarse automáticamente
- Esperará hasta 60 segundos para que se restaure el manifiesto
- Continuará trabajando una vez que se recupere
- No necesitas reiniciar el worker manualmente


In [ ]:
# Setup automático - Crear manifiesto si no existe
import os
from pathlib import Path

print("🔍 Verificando setup inicial...")

# Verificar si ya existe el manifiesto
manifest_path = Path(DRIVE_BASE_PATH) / "manifests" / "batch_manifest.json"

if manifest_path.exists():
    print("✅ Manifiesto ya existe, continuando...")
    print(f"📁 Ubicación: {manifest_path}")
else:
    print("📋 Creando manifiesto de batches automáticamente...")
    print("⚠️ Esto puede tomar unos minutos para 500,000 tareas")

    try:
        # Crear manifiesto
        import subprocess
        import sys

        cmd = [
            sys.executable, "-m", "src.distributed.batch_generator",
            "--drive-path", DRIVE_BASE_PATH,
            "--create-manifest",
            "--batch-size", "50"
        ]

        print(f"🔧 Comando: {' '.join(cmd)}")
        result = subprocess.run(cmd, capture_output=True, text=True)

        if result.returncode == 0:
            print("✅ Manifiesto creado exitosamente")
        else:
            print(f"❌ Error creando manifiesto: {result.stderr}")
            print("💡 Asegúrate de que la carpeta LUMPS_Distributed existe en Drive")

    except Exception as e:
        print(f"❌ Error creando manifiesto: {e}")
        print("💡 Asegúrate de que la carpeta LUMPS_Distributed existe en Drive")

print("🎯 Setup inicial completado")


## 🖥️ Configuración de GPU


## 🔍 Diagnóstico de Estado

Antes de ejecutar el worker, vamos a verificar el estado del sistema:


In [ ]:
# Verificar estado del sistema antes de ejecutar worker
import sys
from pathlib import Path

print("🔍 Verificando estado del sistema...")

# Verificar manifiesto
manifest_path = Path(DRIVE_BASE_PATH) / "manifests" / "batch_manifest.json"
if manifest_path.exists():
    print(f"✅ Manifiesto encontrado: {manifest_path}")

    # Cargar y mostrar estadísticas del manifiesto
    try:
        import json
        with open(manifest_path, 'r') as f:
            manifest = json.load(f)

        total_batches = len(manifest['batches'])
        available = sum(1 for b in manifest['batches'] if b['status'] == 'available')
        locked = sum(1 for b in manifest['batches'] if b['status'] == 'locked')
        completed = sum(1 for b in manifest['batches'] if b['status'] == 'completed')
        failed = sum(1 for b in manifest['batches'] if b['status'] == 'failed')

        print(f"📊 Estado del Manifiesto:")
        print(f"   Total batches: {total_batches}")
        print(f"   Disponibles: {available}")
        print(f"   En progreso: {locked}")
        print(f"   Completados: {completed}")
        print(f"   Fallidos: {failed}")

        if available == 0:
            print("⚠️ ¡No hay batches disponibles!")
            print("💡 Esto significa que:")
            print("   - Todos los batches ya fueron procesados")
            print("   - O todos están siendo procesados por otros workers")
            print("   - El worker se completará inmediatamente")
        else:
            print(f"✅ Hay {available} batches disponibles para procesar")

    except Exception as e:
        print(f"❌ Error leyendo manifiesto: {e}")
else:
    print(f"❌ Manifiesto no encontrado: {manifest_path}")
    print("💡 El worker fallará al intentar ejecutarse")

# Verificar directorios necesarios
dirs_to_check = [
    "manifests",
    "results",
    "logs",
    "locks"
]

print(f"\n📁 Verificando directorios en {DRIVE_BASE_PATH}:")
for dir_name in dirs_to_check:
    dir_path = Path(DRIVE_BASE_PATH) / dir_name
    if dir_path.exists():
        print(f"   ✅ {dir_name}/")
    else:
        print(f"   ❌ {dir_name}/ (no existe)")

print("\n🎯 Diagnóstico completado")


In [ ]:
# Verificar GPU disponible
import torch

print("🖥️ Verificando GPU...")
print(f"   CUDA disponible: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memoria GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print("✅ GPU configurada y lista")
else:
    print("⚠️ No hay GPU disponible, usando CPU")

print(f"   PyTorch versión: {torch.__version__}")


## 🚀 Ejecutar Worker


In [ ]:
# Ejecutar worker distribuido automáticamente
print("🚀 Iniciando worker distribuido...")
print("⚠️ El worker trabajará en loop continuo hasta desconexión")
print("🛑 Presiona el botón de interrumpir para detener")
print("\n" + "="*80)

try:
    # Ejecutar worker con TQDM para progreso visual
    import subprocess
    import sys

    cmd = [
        sys.executable, "generate_dataset_distributed.py",
        "--drive-path", DRIVE_BASE_PATH,
        "--worker-id", WORKER_ID,
        "--verbose"
    ]

    print(f"🔧 Comando: {' '.join(cmd)}")
    print("="*80)

    # Ejecutar comando
    result = subprocess.run(cmd, capture_output=False, text=True)

    if result.returncode != 0:
        print(f"❌ Worker terminó con código: {result.returncode}")
    else:
        print("✅ Worker completado exitosamente")
        print("\n💡 El worker se completó porque:")
        print("   - No hay más batches disponibles para procesar")
        print("   - Todos los batches ya fueron completados por otros workers")
        print("   - O se alcanzó el límite de batches configurado")

except Exception as e:
    print(f"❌ Error ejecutando worker: {e}")
    print("💡 Verifica que el manifiesto existe y tienes permisos de escritura")
    print("🔄 Intentando ejecutar con configuración básica...")

    try:
        cmd_basic = [
            sys.executable, "generate_dataset_distributed.py",
            "--drive-path", DRIVE_BASE_PATH,
            "--worker-id", WORKER_ID
        ]

        print(f"🔧 Comando básico: {' '.join(cmd_basic)}")
        subprocess.run(cmd_basic, capture_output=False, text=True)

    except Exception as e2:
        print(f"❌ Error en comando básico: {e2}")
        print("💡 Revisa la configuración y permisos")
